# Qwen3-8B HR 도메인 파인튜닝

**환경**: Google Colab (T4 15GB / A100 40GB)

**목표**:
- SQL 생성 품질 향상
- 회사 규정 QA 품질 향상

**데이터셋**: 200개 (SQL 100 + RAG 100)

## 1. 환경 설정

In [ ]:
# Unsloth 설치 (Colab 최적화)
%%capture
!pip install unsloth
!pip install --no-deps trl peft accelerate bitsandbytes

In [ ]:
# GPU 확인
!nvidia-smi

## 2. 데이터셋 업로드

GitHub에서 직접 다운로드하거나 파일 업로드

In [ ]:
# 방법 1: GitHub Raw URL에서 다운로드 (repo가 public인 경우)
# !wget -O combined_train.json "https://raw.githubusercontent.com/K-tuna/enterprise-hr-agent/main/data/finetuning/combined_train.json"

# 방법 2: Google Drive에서 마운트
from google.colab import drive
drive.mount('/content/drive')

# Drive에 combined_train.json 업로드 후 경로 지정
DATASET_PATH = "/content/drive/MyDrive/enterprise-hr-agent/combined_train.json"

# 방법 3: 직접 업로드
# from google.colab import files
# uploaded = files.upload()  # combined_train.json 선택
# DATASET_PATH = "combined_train.json"

In [ ]:
# 데이터셋 확인
import json

with open(DATASET_PATH, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

print(f"총 데이터 수: {len(raw_data)}개")
print(f"\n첫 번째 예시:")
print(json.dumps(raw_data[0], ensure_ascii=False, indent=2))

## 3. 모델 로드

In [ ]:
from unsloth import FastLanguageModel
import torch

# 설정
max_seq_length = 2048
lora_rank = 32  # 8GB에서는 16, 충분하면 32

# 모델 로드 (4-bit 양자화)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen3-8B",  # 또는 "Qwen/Qwen2.5-7B-Instruct"
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,  # 자동 감지
)

print(f"모델 로드 완료: {model.config.model_type}")

In [ ]:
# LoRA 어댑터 설정
model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=lora_rank * 2,  # 학습 속도 향상
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    max_seq_length=max_seq_length,
)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"학습 가능 파라미터: {trainable_params:,}")

## 4. 데이터셋 준비

In [ ]:
from datasets import Dataset
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

# Tokenizer에 chat template 적용
tokenizer = get_chat_template(
    tokenizer,
    chat_template="qwen-2.5",  # Qwen3도 동일
)

# 포맷팅 함수
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo, 
            tokenize=False, 
            add_generation_prompt=False
        ) 
        for convo in convos
    ]
    return {"text": texts}

# Dataset 생성
dataset = Dataset.from_list(raw_data)
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched=True)

print(f"\n포맷팅된 첫 번째 예시:")
print(dataset[0]["text"][:500])

## 5. 학습

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,  # 짧은 시퀀스 패킹으로 효율성 향상
    args=TrainingArguments(
        output_dir="./outputs",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        learning_rate=2e-4,
        warmup_steps=10,
        logging_steps=10,
        save_steps=100,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        report_to="none",
    ),
)

In [ ]:
# 학습 시작
print("=" * 50)
print("학습 시작...")
print("=" * 50)

trainer_stats = trainer.train()

print("\n학습 완료!")
print(f"총 학습 시간: {trainer_stats.metrics['train_runtime']:.2f}초")
print(f"최종 Loss: {trainer_stats.metrics['train_loss']:.4f}")

## 6. 테스트

In [ ]:
# 추론 모드 활성화
FastLanguageModel.for_inference(model)

# 테스트 질문
test_questions = [
    "개발팀 평균 급여는?",
    "연차휴가 며칠이야?",
    "김철수 평가 점수 알려줘",
    "재택근무 가능해?",
]

for question in test_questions:
    messages = [{"role": "user", "content": question}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)
    
    outputs = model.generate(
        inputs,
        max_new_tokens=256,
        temperature=0.1,
        do_sample=True,
    )
    
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    print(f"Q: {question}")
    print(f"A: {response}")
    print("-" * 50)

## 7. GGUF 변환 및 저장

In [ ]:
# GGUF 변환 (Ollama용)
model.save_pretrained_gguf(
    "qwen3-hr",
    tokenizer,
    quantization_method="q4_k_m",  # 4-bit 양자화 (품질/크기 균형)
)

print("GGUF 변환 완료!")
!ls -lh qwen3-hr*

In [ ]:
# Google Drive에 저장
import shutil

SAVE_DIR = "/content/drive/MyDrive/enterprise-hr-agent/models"
!mkdir -p "{SAVE_DIR}"

# GGUF 파일 복사
!cp qwen3-hr-q4_k_m.gguf "{SAVE_DIR}/"

print(f"\n저장 완료: {SAVE_DIR}")
!ls -lh "{SAVE_DIR}"

## 8. 로컬 배포 가이드

### Google Drive에서 다운로드
1. Google Drive에서 `qwen3-hr-q4_k_m.gguf` 파일 다운로드
2. `models/` 디렉토리에 저장

### Ollama 등록
```bash
# Modelfile 생성
echo 'FROM ./qwen3-hr-q4_k_m.gguf
TEMPLATE """{{ if .System }}<|im_start|>system
{{ .System }}<|im_end|>
{{ end }}<|im_start|>user
{{ .Prompt }}<|im_end|>
<|im_start|>assistant
"""' > Modelfile

# Ollama에 등록
ollama create qwen3-hr -f Modelfile

# 테스트
ollama run qwen3-hr "개발팀 평균 급여 SQL 작성해줘"
```

In [ ]:
print("파인튜닝 완료!")
print("\n다음 단계:")
print("1. Google Drive에서 GGUF 파일 다운로드")
print("2. 로컬에서 Ollama에 등록")
print("3. 기존 qwen3:8b와 성능 비교")